# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"s

gpu_num = 2

os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

In [2]:
!echo $CUDA_VISIBLE_DEVICES

2


In [5]:
### IJB-C Dataset
# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset
num_classes = 85742 
dataset_len = 5822653
batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size

w_decay=5e-4
epochs = 100

save_steps = 1000
steps = 1
is_ccrop=False
binary_img=True

is_Adam = False   # True
projection_head = False  # True
dgx = True

head_type = 'CadHead' # ''ArcHead', CosHead', 'SphereHead', 'CurHead', 'AdaHead', CadHead'  
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception, MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
# Backbones w/o pretrained weights:
#      MnasNetA1, MnasNetB1, MnasNetSmall 
backbone_type = 'EfficientNetB6' 

if head_type == 'SphereHead':
    base_lr = 0.01 
    margin = 1.35
    logist_scale = 30.0 
elif head_type == 'CosHead':
    base_lr = 0.01 
    margin=0.35
    logist_scale=64
elif head_type == 'ArcHead':
    base_lr = 0.1 
    margin=0.5
    logist_scale=64
elif head_type == 'CurHead': 
    base_lr = 0.1 
    margin=0.5
    logist_scale=64
elif head_type == 'CadHead': 
    base_lr = 0.1 
    margin=0.0
    logist_scale=64
elif head_type == 'AdaHead':
    base_lr = 0.1 
    margin=0.0
    logist_scale=64
else:
    base_lr = 0.01 # initially 0.01
    
print(head_type)
print(backbone_type)
print("projection head:", projection_head)
print("Adam:", is_Adam)
print("epoch:", epochs)
print("batch size:", batch_size)

CadHead
EfficientNetB6
projection head: False
Adam: False
epoch: 100
batch size: 128


In [6]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

# print("data: ", train_dataset)


strategy = tf.distribute.MirroredStrategy(devices=[f"/gpu:{gpu_num}"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             margin=margin, 
                             logist_scale=logist_scale,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True,
                             projection_head=projection_head)
    model.summary()

    learning_rate = tf.constant(base_lr)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                                 decay_steps=300000, 
                                                   decay_rate=0.1,staircase=True)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule)
    else:
#         optimizer = tf.keras.optimizers.SGD(
#             learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb6 (Functional)     (None, 4, 4, 2304)   40960136    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          18886144    efficientnetb6[0][0]             
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [ ]:
from pathlib import Path
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/archead',
                                  update_freq = 1000,
                                  profile_batch=2)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# callbacks = [mc_callback, tb_callback, early_stopping]
callbacks = [mc_callback, early_stopping]
# callbacks = [mc_callback, tb_callback, early_stopping, lr_scheduler]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/100
 1000/36391 [..............................] - ETA: 4:57:10 - loss: 12.8414 - s: 16.7481 - correct_cos_mean: 0.0165
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_1_l_12.851656913757324.ckpt
 2000/36391 [>.............................] - ETA: 4:44:54 - loss: 12.7370 - s: 17.0700 - correct_cos_mean: 0.0436
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_1_l_12.408902168273926.ckpt
 3000/36391 [=>............................] - ETA: 4:34:12 - loss: 12.5557 - s: 17.4174 - correct_cos_mean: 0.0712
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_1_l_11.979085922241211.ckpt
 4000/36391 [==>...........................] - ETA: 4:24:46 - loss: 12.3585 - s: 17.7830 - correct_cos_mean: 0.0998
Epoch 00001: saving model to /raid/wor

31000/36391 [========================>.....] - ETA: 44:16 - loss: 8.4434 - s: 21.7188 - correct_cos_mean: 0.4655
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_1_l_5.498032093048096.ckpt
32000/36391 [=========================>....] - ETA: 36:04 - loss: 8.3496 - s: 21.7803 - correct_cos_mean: 0.4716
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_1_l_5.389311790466309.ckpt
33000/36391 [==========================>...] - ETA: 27:52 - loss: 8.2583 - s: 21.8387 - correct_cos_mean: 0.4775
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_1_l_5.284886837005615.ckpt
34000/36391 [===========================>..] - ETA: 19:38 - loss: 8.1694 - s: 21.8939 - correct_cos_mean: 0.4832
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognit

23609/36391 [==================>...........] - ETA: 1:46:36 - loss: 1.5709 - s: 21.4531 - correct_cos_mean: 0.7197
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_2_l_1.4930170774459839.ckpt
24609/36391 [===================>..........] - ETA: 1:38:17 - loss: 1.5676 - s: 21.3805 - correct_cos_mean: 0.7210
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_2_l_1.488429307937622.ckpt
25609/36391 [====================>.........] - ETA: 1:29:57 - loss: 1.5644 - s: 21.3086 - correct_cos_mean: 0.7222
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_2_l_1.4840303659439087.ckpt
26609/36391 [====================>.........] - ETA: 1:21:36 - loss: 1.5613 - s: 21.2392 - correct_cos_mean: 0.7234
Epoch 00002: saving model to /raid/workspace/honghee/Fa

16218/36391 [============>.................] - ETA: 2:45:07 - loss: 1.3075 - s: 18.2505 - correct_cos_mean: 0.7770
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_3_l_1.2959342002868652.ckpt
17218/36391 [=============>................] - ETA: 2:36:57 - loss: 1.3068 - s: 18.2272 - correct_cos_mean: 0.7774
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_3_l_1.2947567701339722.ckpt
18218/36391 [==============>...............] - ETA: 2:28:46 - loss: 1.3061 - s: 18.2038 - correct_cos_mean: 0.7778
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_3_l_1.2939825057983398.ckpt
19218/36391 [==============>...............] - ETA: 2:20:35 - loss: 1.3055 - s: 18.1796 - correct_cos_mean: 0.7782
Epoch 00003: saving model to /raid/workspace/honghee/F

 8827/36391 [======>.......................] - ETA: 3:45:16 - loss: 1.2348 - s: 17.0920 - correct_cos_mean: 0.7982
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_4_l_1.2311584949493408.ckpt
 9827/36391 [=======>......................] - ETA: 3:37:02 - loss: 1.2345 - s: 17.0774 - correct_cos_mean: 0.7984
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_4_l_1.2306337356567383.ckpt
10827/36391 [=======>......................] - ETA: 3:28:58 - loss: 1.2341 - s: 17.0651 - correct_cos_mean: 0.7987
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_4_l_1.2298989295959473.ckpt
11827/36391 [========>.....................] - ETA: 3:20:48 - loss: 1.2337 - s: 17.0548 - correct_cos_mean: 0.7988
Epoch 00004: saving model to /raid/workspace/honghee/F

...

 2744/36391 [=>............................] - ETA: 4:30:52 - loss: 1.0676 - s: 14.0195 - correct_cos_mean: 0.8723
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_17_l_1.0665467977523804.ckpt
 3744/36391 [==>...........................] - ETA: 4:22:50 - loss: 1.0672 - s: 14.0177 - correct_cos_mean: 0.8725
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_17_l_1.0658314228057861.ckpt
 4744/36391 [==>...........................] - ETA: 4:14:56 - loss: 1.0669 - s: 14.0169 - correct_cos_mean: 0.8725
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_17_l_1.0653820037841797.ckpt
 5744/36391 [===>..........................] - ETA: 4:07:01 - loss: 1.0666 - s: 14.0147 - correct_cos_mean: 0.8726
Epoch 00017: saving model to /raid/workspace/honghe

32744/36391 [=========================>....] - ETA: 29:27 - loss: 1.0631 - s: 13.9816 - correct_cos_mean: 0.8740
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_17_l_1.061065673828125.ckpt
33744/36391 [==========================>...] - ETA: 21:22 - loss: 1.0631 - s: 13.9803 - correct_cos_mean: 0.8741
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_17_l_1.0609771013259888.ckpt
34744/36391 [===========================>..] - ETA: 13:18 - loss: 1.0630 - s: 13.9792 - correct_cos_mean: 0.8741
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_17_l_1.0608901977539062.ckpt
35744/36391 [============================>.] - ETA: 5:13 - loss: 1.0630 - s: 13.9781 - correct_cos_mean: 0.8742
Epoch 00017: saving model to /raid/workspace/honghee/FaceReco

25353/36391 [===================>..........] - ETA: 1:29:04 - loss: 1.0587 - s: 13.9171 - correct_cos_mean: 0.8765
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_18_l_1.0569655895233154.ckpt
26353/36391 [====================>.........] - ETA: 1:21:00 - loss: 1.0586 - s: 13.9161 - correct_cos_mean: 0.8765
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_18_l_1.0568044185638428.ckpt
27353/36391 [=====================>........] - ETA: 1:12:55 - loss: 1.0586 - s: 13.9148 - correct_cos_mean: 0.8766
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_18_l_1.0566644668579102.ckpt
28353/36391 [======================>.......] - ETA: 1:04:51 - loss: 1.0585 - s: 13.9137 - correct_cos_mean: 0.8766
Epoch 00018: saving model to /raid/workspace/honghe

17962/36391 [=============>................] - ETA: 2:27:46 - loss: 1.0539 - s: 13.8584 - correct_cos_mean: 0.8788
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_19_l_1.0522161722183228.ckpt
18962/36391 [==============>...............] - ETA: 2:19:44 - loss: 1.0538 - s: 13.8571 - correct_cos_mean: 0.8788
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_19_l_1.052077054977417.ckpt
19962/36391 [===============>..............] - ETA: 2:11:41 - loss: 1.0537 - s: 13.8559 - correct_cos_mean: 0.8789
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_19_l_1.052122950553894.ckpt
20962/36391 [================>.............] - ETA: 2:03:39 - loss: 1.0537 - s: 13.8554 - correct_cos_mean: 0.8789
Epoch 00019: saving model to /raid/workspace/honghee/

10571/36391 [=======>......................] - ETA: 3:26:09 - loss: 1.0513 - s: 13.8029 - correct_cos_mean: 0.8809
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_20_l_1.0484706163406372.ckpt
11571/36391 [========>.....................] - ETA: 3:18:12 - loss: 1.0510 - s: 13.8029 - correct_cos_mean: 0.8809
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_20_l_1.0486202239990234.ckpt
12571/36391 [=========>....................] - ETA: 3:10:11 - loss: 1.0508 - s: 13.8023 - correct_cos_mean: 0.8810
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CadHead_check/SGD/e_20_l_1.0484004020690918.ckpt
13571/36391 [==========>...................] - ETA: 3:02:10 - loss: 1.0506 - s: 13.8015 - correct_cos_mean: 0.8810
Epoch 00020: saving model to /raid/workspace/honghe

### Resume training with latest checkpoint

In [4]:
from glob import glob
from pathlib import Path

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
file_list = []
for files in glob(f'{base_dir}{save_name}'):
    if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
    file_list.append( loss  )
file_list.sort()

load_file_name = []
for files in glob(f'{base_dir}{save_name}'):
    if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        pass
    elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

2
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_19.344934463500977.ckpt


In [5]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{gpu_num}"])
strategy = tf.distribute.MirroredStrategy(devices=[f"/gpu:{gpu_num}"])

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True,
                             projection_head=projection_head)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                                 decay_steps=300000, 
                                                   decay_rate=0.1,staircase=True)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule)
    else:
#         optimizer = tf.keras.optimizers.SGD(
#             learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/curhead',
                                  update_freq = 1000,
                                  profile_batch=2)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

callbacks = [mc_callback, early_stopping]
# callbacks = [mc_callback, tb_callback, early_stopping]
# callbacks = [mc_callback, tb_callback]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 4, 4, 2048)   23587712    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          16787968    resnet50[0][0]                   
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [6]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 3/20
 1000/36391 [..............................] - ETA: 1:19:53 - loss: 17.1177
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.141321182250977.ckpt
 2000/36391 [>.............................] - ETA: 1:16:17 - loss: 17.1267
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.109325408935547.ckpt
 3000/36391 [=>............................] - ETA: 1:13:15 - loss: 17.1130
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.05596923828125.ckpt
 4000/36391 [==>...........................] - ETA: 1:08:11 - loss: 17.0947
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.030012130737305.ckpt
 5000/36391 [===>..........................] - ETA: 1:04:03 - 

36391/36391 [==============================] - 4335s 119ms/step - loss: 16.0488 - val_loss: 13.2359
Epoch 4/20
  609/36391 [..............................] - ETA: 1:05:35 - loss: 11.5173
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_4_l_11.468496322631836.ckpt
 1609/36391 [>.............................] - ETA: 1:04:13 - loss: 11.5025
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_4_l_11.478160858154297.ckpt
 2609/36391 [=>............................] - ETA: 1:02:11 - loss: 11.4803
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_4_l_11.414444923400879.ckpt
 3609/36391 [=>............................] - ETA: 59:35 - loss: 11.4551
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_che

35218/36391 [============================>.] - ETA: 2:06 - loss: 7.8365
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_5_l_7.4903645515441895.ckpt
36218/36391 [============================>.] - ETA: 18s - loss: 7.8267
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_5_l_7.470621585845947.ckpt
36391/36391 [==============================] - 4223s 116ms/step - loss: 7.8250 - val_loss: 7.2360
Epoch 6/20
  827/36391 [..............................] - ETA: 1:06:12 - loss: 6.2054
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_6_l_6.207218647003174.ckpt
 1827/36391 [>.............................] - ETA: 1:03:24 - loss: 6.2195
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_6_l_

32436/36391 [=========================>....] - ETA: 7:00 - loss: 5.2989
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.252306938171387.ckpt
33436/36391 [==========================>...] - ETA: 5:13 - loss: 5.2974
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.244982719421387.ckpt
34436/36391 [===========================>..] - ETA: 3:27 - loss: 5.2957
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.237680912017822.ckpt
35436/36391 [============================>.] - ETA: 1:41 - loss: 5.2940
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.2306084632873535.ckpt
36391/36391 [==============================] - 4158s 114ms/step - loss: 5.2923 - val_loss: 

32045/36391 [=========================>....] - ETA: 7:42 - loss: 4.6699
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.609582901000977.ckpt
33045/36391 [==========================>...] - ETA: 5:56 - loss: 4.6676
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.577266216278076.ckpt
34045/36391 [===========================>..] - ETA: 4:09 - loss: 4.6644
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.544215202331543.ckpt
35045/36391 [===========================>..] - ETA: 2:23 - loss: 4.6605
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.5117692947387695.ckpt
36045/36391 [============================>.] - ETA: 36s - loss: 4.6560
Epoch 00008: saving 

...

26481/36391 [====================>.........] - ETA: 17:59 - loss: 2.6501
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.701799154281616.ckpt
27481/36391 [=====================>........] - ETA: 16:09 - loss: 2.6519
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.6978394985198975.ckpt
28481/36391 [======================>.......] - ETA: 14:21 - loss: 2.6534
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.6940553188323975.ckpt
29481/36391 [=======================>......] - ETA: 12:31 - loss: 2.6548
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.6954338550567627.ckpt
30481/36391 [========================>.....] - ETA: 10:43 - loss: 2.6562
Epoch 00

23699/36391 [==================>...........] - ETA: 22:57 - loss: 2.5315
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.6026577949523926.ckpt
24699/36391 [===================>..........] - ETA: 21:07 - loss: 2.5343
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.6009271144866943.ckpt
25699/36391 [====================>.........] - ETA: 19:18 - loss: 2.5369
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.5991830825805664.ckpt
26699/36391 [=====================>........] - ETA: 17:29 - loss: 2.5392
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.5963375568389893.ckpt
27699/36391 [=====================>........] - ETA: 15:41 - loss: 2.5412
Epoch 0

20917/36391 [================>.............] - ETA: 27:35 - loss: 2.4218
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.50701642036438.ckpt
21917/36391 [=================>............] - ETA: 25:47 - loss: 2.4257
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.507241725921631.ckpt
22917/36391 [=================>............] - ETA: 24:00 - loss: 2.4292
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.507878065109253.ckpt
23917/36391 [==================>...........] - ETA: 22:15 - loss: 2.4325
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.5082669258117676.ckpt
24917/36391 [===================>..........] - ETA: 20:27 - loss: 2.4355
Epoch 00016

18135/36391 [=============>................] - ETA: 32:32 - loss: 2.1179
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.223524332046509.ckpt
19135/36391 [==============>...............] - ETA: 30:44 - loss: 2.1234
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.223574161529541.ckpt
20135/36391 [===============>..............] - ETA: 28:59 - loss: 2.1283
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.2222647666931152.ckpt
21135/36391 [================>.............] - ETA: 27:12 - loss: 2.1328
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.222892999649048.ckpt
22135/36391 [=================>............] - ETA: 25:26 - loss: 2.1369
Epoch 0001

15353/36391 [===========>..................] - ETA: 37:57 - loss: 2.0341
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.150834321975708.ckpt
16353/36391 [============>.................] - ETA: 36:10 - loss: 2.0413
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.15371036529541.ckpt
17353/36391 [=============>................] - ETA: 34:23 - loss: 2.0479
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.157562017440796.ckpt
18353/36391 [==============>...............] - ETA: 32:36 - loss: 2.0540
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.1605777740478516.ckpt
19353/36391 [==============>...............] - ETA: 30:51 - loss: 2.0595
Epoch 00020